<a href="https://colab.research.google.com/github/MooinBoanBangwidae/mail/blob/main/%EC%9D%B4%EB%A9%94%EC%9D%BC_%EC%A0%84%EC%86%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import torch
from IPython.display import Image, clear_output

In [ ]:
#yolov5 다운
!git clone https://github.com/ultralytics/yolov5.git

%cd /content/yolov5/
!pip install -r requirements.txt

fatal: destination path 'yolov5' already exists and is not an empty directory.
/content/yolov5
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
%cd ..

/


In [ ]:
import os
print(os.getcwd())
!ls

# 실행시 등장하는 URL을 클릭하여 허용해주면 인증KEY가 나타난다. 복사하여 URL아래 빈칸에 붙여넣으면 마운트에 성공하게된다.
from google.colab import drive
drive.mount('./MyDrive')

# 마운트된 내 드라이브를 확인해보자
!ls

/
bin	 dev   lib32   mnt			 python-apt  srv    usr
boot	 etc   lib64   NGC-DL-CONTAINER-LICENSE  root	     sys    var
content  home  libx32  opt			 run	     tmp
datalab  lib   media   proc			 sbin	     tools
Mounted at ./MyDrive
bin	 dev   lib32   mnt			 proc	     sbin  tools
boot	 etc   lib64   MyDrive			 python-apt  srv   usr
content  home  libx32  NGC-DL-CONTAINER-LICENSE  root	     sys   var
datalab  lib   media   opt			 run	     tmp


In [ ]:
pwd

'/'

In [ ]:
import sys

# assuming yolov5 directory is in the current directory
sys.path.append('./yolov5')

from models.experimental import attempt_load

# load the model
model = attempt_load('/MyDrive/MyDrive/yolov5s_results/weights/best.pt')

Fusing layers... 
YOLOv5s summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs


In [ ]:
vid_path = '/MyDrive/MyDrive/KakaoTalk_20230510_155004310.mp4'

In [ ]:
#이메일 로그인 계정 입력
sender = 'fgh2585@naver.com'
password = 'vhrlgkwlak!'

#수신자 이메일 입력
receiver = 'fgh2585@gmail.com'

def smtp_setting(type, email, password):
  mail_type = None
  port = 587

  if type == 'naver':
    mail_type = 'smtp.naver.com'
  elif type == 'gmail':
    mail_type = 'smtp.gmail.com'
  else:
    mail_type = 'smtp.gmail.com'

  # SMTP 세션 생성
  smtp = smtplib.SMTP(mail_type, port)
  smtp.set_debuglevel(True)

  # SMTP 계정 인증 설정
  smtp.ehlo()
  smtp.starttls() # TLS 사용시 호출
  smtp.login(email, password) # 로그인

  return smtp

In [ ]:
def send_plain_mail(sender, receiver, email, password, subject, content):
  try:
    # SMTP 세션 생성
    smtp = smtp_setting('naver', email, password)

    # 이메일 데이터 설정
    msg = MIMEText(content)
    msg['Subject'] = subject
    msg['From'] = sender  # 발신자
    msg['To'] = receiver  # 수신자

    # 메일 전송
    smtp.sendmail(sender, receiver, msg.as_string())
  except Exception as e:
    print('error', e)
  finally:
    if smtp is not None:
      smtp.quit()

In [ ]:
import cv2
import torch
import numpy as np
import smtplib
from email.message import EmailMessage
from utils.general import non_max_suppression

# Load video
vidcap = cv2.VideoCapture(vid_path)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)

# Run inference
success, image = vidcap.read()

def scale_coords(img1_shape, coords, img0_shape):
    # Normalizes coords (inplace) from img1_shape to img0_shape, resizing bounding boxes from img1_shape to img0_shape
    gain = min(img0_shape[0] / img1_shape[0], img0_shape[1] / img1_shape[1])  # gain  = old / new
    pad = (img0_shape[1] - img1_shape[1] * gain) / 2, (img0_shape[0] - img1_shape[0] * gain) / 2  # wh padding
    coords[:, [0, 2]] -= pad[0]  # x padding
    coords[:, [1, 3]] -= pad[1]  # y padding
    coords[:, :4] /= gain
    return coords.round()

while success:
    # Resize and normalize image for input
    image = cv2.resize(image, (640, 640))
    image = image / 255.0

    # Add an extra dimension for batch
    image = np.expand_dims(image, axis=0)

    # Swap the channel dimension
    image = np.transpose(image, (0, 3, 1, 2))

    # Convert to PyTorch tensor and ensure the type is float
    image = torch.from_numpy(image).float().to(device)

    # Pass image through model
    pred = model(image)[0]

    # Apply NMS
    pred = non_max_suppression(pred, 0.4, 0.5, classes=0, agnostic=False)

    # Process results
    for i, det in enumerate(pred):  # detections for image
        if len(det):
            # Rescale boxes from img_size to original size
            det[:, :4] = scale_coords(image.shape[2:], det[:, :4], image.shape).round()

            # Print results
            for *xyxy, conf, cls in reversed(det):
                if cls == 0:  # If 'thief' class
                    print('Thief detected!')

                    # Set up the email
                    send_plain_mail(sender, receiver, sender, password, 'Thief Detection Alert', '매장에 손님의 도둑질 행위가 감지 되었습니다.')

    success, image = vidcap.read()

Thief detected!


send: 'ehlo [172.28.0.12]\r\n'
reply: b'250-smtp.naver.com Pleased to meet you\r\n'
reply: b'250-SIZE 20971520\r\n'
reply: b'250-8BITMIME\r\n'
reply: b'250-PIPELINING\r\n'
reply: b'250-STARTTLS\r\n'
reply: b'250-AUTH PLAIN LOGIN\r\n'
reply: b'250 ENHANCEDSTATUSCODES\r\n'
reply: retcode (250); Msg: b'smtp.naver.com Pleased to meet you\nSIZE 20971520\n8BITMIME\nPIPELINING\nSTARTTLS\nAUTH PLAIN LOGIN\nENHANCEDSTATUSCODES'
send: 'STARTTLS\r\n'
reply: b'220 2.0.0 Ready to start TLS XOJKTibAQqmGp+0vZkkH2g - nsmtp\r\n'
reply: retcode (220); Msg: b'2.0.0 Ready to start TLS XOJKTibAQqmGp+0vZkkH2g - nsmtp'
send: 'ehlo [172.28.0.12]\r\n'
reply: b'250-smtp.naver.com Pleased to meet you\r\n'
reply: b'250-SIZE 20971520\r\n'
reply: b'250-8BITMIME\r\n'
reply: b'250-PIPELINING\r\n'
reply: b'250-AUTH PLAIN LOGIN\r\n'
reply: b'250 ENHANCEDSTATUSCODES\r\n'
reply: retcode (250); Msg: b'smtp.naver.com Pleased to meet you\nSIZE 20971520\n8BITMIME\nPIPELINING\nAUTH PLAIN LOGIN\nENHANCEDSTATUSCODES'
send: 'AUT

Thief detected!


send: 'ehlo [172.28.0.12]\r\n'
reply: b'250-smtp.naver.com Pleased to meet you\r\n'
reply: b'250-SIZE 20971520\r\n'
reply: b'250-8BITMIME\r\n'
reply: b'250-PIPELINING\r\n'
reply: b'250-STARTTLS\r\n'
reply: b'250-AUTH PLAIN LOGIN\r\n'
reply: b'250 ENHANCEDSTATUSCODES\r\n'
reply: retcode (250); Msg: b'smtp.naver.com Pleased to meet you\nSIZE 20971520\n8BITMIME\nPIPELINING\nSTARTTLS\nAUTH PLAIN LOGIN\nENHANCEDSTATUSCODES'
send: 'STARTTLS\r\n'
reply: b'220 2.0.0 Ready to start TLS ybcRpzhbQxakEXm00g2a7A - nsmtp\r\n'
reply: retcode (220); Msg: b'2.0.0 Ready to start TLS ybcRpzhbQxakEXm00g2a7A - nsmtp'
send: 'ehlo [172.28.0.12]\r\n'
reply: b'250-smtp.naver.com Pleased to meet you\r\n'
reply: b'250-SIZE 20971520\r\n'
reply: b'250-8BITMIME\r\n'
reply: b'250-PIPELINING\r\n'
reply: b'250-AUTH PLAIN LOGIN\r\n'
reply: b'250 ENHANCEDSTATUSCODES\r\n'
reply: retcode (250); Msg: b'smtp.naver.com Pleased to meet you\nSIZE 20971520\n8BITMIME\nPIPELINING\nAUTH PLAIN LOGIN\nENHANCEDSTATUSCODES'
send: 'AUT

Thief detected!


...

send: 'ehlo [172.28.0.12]\r\n'
reply: b'250-smtp.naver.com Pleased to meet you\r\n'
reply: b'250-SIZE 20971520\r\n'
reply: b'250-8BITMIME\r\n'
reply: b'250-PIPELINING\r\n'
reply: b'250-STARTTLS\r\n'
reply: b'250-AUTH PLAIN LOGIN\r\n'
reply: b'250 ENHANCEDSTATUSCODES\r\n'
reply: retcode (250); Msg: b'smtp.naver.com Pleased to meet you\nSIZE 20971520\n8BITMIME\nPIPELINING\nSTARTTLS\nAUTH PLAIN LOGIN\nENHANCEDSTATUSCODES'
send: 'STARTTLS\r\n'
reply: b'220 2.0.0 Ready to start TLS 4D4ZpSf1TTCGUiDNONgNtQ - nsmtp\r\n'
reply: retcode (220); Msg: b'2.0.0 Ready to start TLS 4D4ZpSf1TTCGUiDNONgNtQ - nsmtp'
send: 'ehlo [172.28.0.12]\r\n'
reply: b'250-smtp.naver.com Pleased to meet you\r\n'
reply: b'250-SIZE 20971520\r\n'
reply: b'250-8BITMIME\r\n'
reply: b'250-PIPELINING\r\n'
reply: b'250-AUTH PLAIN LOGIN\r\n'
reply: b'250 ENHANCEDSTATUSCODES\r\n'
reply: retcode (250); Msg: b'smtp.naver.com Pleased to meet you\nSIZE 20971520\n8BITMIME\nPIPELINING\nAUTH PLAIN LOGIN\nENHANCEDSTATUSCODES'
send: 'AUT

Thief detected!


send: 'ehlo [172.28.0.12]\r\n'
reply: b'250-smtp.naver.com Pleased to meet you\r\n'
reply: b'250-SIZE 20971520\r\n'
reply: b'250-8BITMIME\r\n'
reply: b'250-PIPELINING\r\n'
reply: b'250-STARTTLS\r\n'
reply: b'250-AUTH PLAIN LOGIN\r\n'
reply: b'250 ENHANCEDSTATUSCODES\r\n'
reply: retcode (250); Msg: b'smtp.naver.com Pleased to meet you\nSIZE 20971520\n8BITMIME\nPIPELINING\nSTARTTLS\nAUTH PLAIN LOGIN\nENHANCEDSTATUSCODES'
send: 'STARTTLS\r\n'
reply: b'220 2.0.0 Ready to start TLS IhwWJhJtQd6CRgBcByyj8g - nsmtp\r\n'
reply: retcode (220); Msg: b'2.0.0 Ready to start TLS IhwWJhJtQd6CRgBcByyj8g - nsmtp'
send: 'ehlo [172.28.0.12]\r\n'
reply: b'250-smtp.naver.com Pleased to meet you\r\n'
reply: b'250-SIZE 20971520\r\n'
reply: b'250-8BITMIME\r\n'
reply: b'250-PIPELINING\r\n'
reply: b'250-AUTH PLAIN LOGIN\r\n'
reply: b'250 ENHANCEDSTATUSCODES\r\n'
reply: retcode (250); Msg: b'smtp.naver.com Pleased to meet you\nSIZE 20971520\n8BITMIME\nPIPELINING\nAUTH PLAIN LOGIN\nENHANCEDSTATUSCODES'
send: 'AUT

Thief detected!


send: 'ehlo [172.28.0.12]\r\n'
reply: b'250-smtp.naver.com Pleased to meet you\r\n'
reply: b'250-SIZE 20971520\r\n'
reply: b'250-8BITMIME\r\n'
reply: b'250-PIPELINING\r\n'
reply: b'250-STARTTLS\r\n'
reply: b'250-AUTH PLAIN LOGIN\r\n'
reply: b'250 ENHANCEDSTATUSCODES\r\n'
reply: retcode (250); Msg: b'smtp.naver.com Pleased to meet you\nSIZE 20971520\n8BITMIME\nPIPELINING\nSTARTTLS\nAUTH PLAIN LOGIN\nENHANCEDSTATUSCODES'
send: 'STARTTLS\r\n'
reply: b'220 2.0.0 Ready to start TLS 0JSH1ZoQSFOKACS4oxnCBg - nsmtp\r\n'
reply: retcode (220); Msg: b'2.0.0 Ready to start TLS 0JSH1ZoQSFOKACS4oxnCBg - nsmtp'
send: 'ehlo [172.28.0.12]\r\n'
reply: b'250-smtp.naver.com Pleased to meet you\r\n'
reply: b'250-SIZE 20971520\r\n'
reply: b'250-8BITMIME\r\n'
reply: b'250-PIPELINING\r\n'
reply: b'250-AUTH PLAIN LOGIN\r\n'
reply: b'250 ENHANCEDSTATUSCODES\r\n'
reply: retcode (250); Msg: b'smtp.naver.com Pleased to meet you\nSIZE 20971520\n8BITMIME\nPIPELINING\nAUTH PLAIN LOGIN\nENHANCEDSTATUSCODES'
send: 'AUT